In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [2]:
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import matplotlib.pyplot as plt
import numpy as np
import os

asic cnn<br>
Initialising the CNN

In [4]:
classifier = Sequential()

Step 1 - Convolution

In [5]:
classifier.add(Conv2D(32, (3, 3), input_shape = (128, 128, 3), activation = 'relu'))

Step 2 - Pooling

In [6]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Adding a second convolutional layer

In [7]:
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Step 3 - Flattening

In [8]:
classifier.add(Flatten())

Step 4 - Full connection

In [9]:
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 10, activation = 'sigmoid'))

Compiling the CNN

In [10]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

In [12]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [13]:
training_set = train_datagen.flow_from_directory('Dataset/train', # relative path from working directoy
                                                 target_size = (128, 128),
                                                 batch_size = 6, class_mode = 'categorical')
valid_set = test_datagen.flow_from_directory('Dataset/val', # relative path from working directoy
                                             target_size = (128, 128), 
                                        batch_size = 3, class_mode = 'categorical')

Found 143 images belonging to 10 classes.
Found 53 images belonging to 10 classes.


In [14]:
labels = (training_set.class_indices)
print(labels)

{'Tomato - Bacterial_spot': 0, 'Tomato - Early_blight': 1, 'Tomato - Healthy': 2, 'Tomato - Late_blight': 3, 'Tomato - Leaf_Mold': 4, 'Tomato - Septoria_leaf_spot': 5, 'Tomato - Target_Spot': 6, 'Tomato - Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato - Tomato_mosaic_virus': 8, 'Tomato - Two-spotted_spider_mite': 9}


In [15]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 20,
                         epochs = 50,
                         validation_data=valid_set
                         )

C:\Users\Acer\.conda\envs\leaf-disease\lib\site-packages\tensorflow\python\keras\engine\training_v1.py:1240: UserWarning: `model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`model.fit_generator` is deprecated and '


Epoch 1/50
20/20 [==============================] - ETA: 0s - batch: 9.5000 - size: 5.9500 - loss: 2.3609 - accuracy: 0.1176

C:\Users\Acer\.conda\envs\leaf-disease\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


20/20 [==============================] - 5s 198ms/step - batch: 9.5000 - size: 5.9500 - loss: 2.3609 - accuracy: 0.1176 - val_loss: 2.2955 - val_accuracy: 0.1321
Epoch 2/50
20/20 [==============================] - 4s 207ms/step - batch: 9.5000 - size: 6.0000 - loss: 2.2603 - accuracy: 0.1500 - val_loss: 2.2807 - val_accuracy: 0.1321
Epoch 3/50
20/20 [==============================] - 4s 177ms/step - batch: 9.5000 - size: 5.9500 - loss: 2.1618 - accuracy: 0.2101 - val_loss: 2.3487 - val_accuracy: 0.2075
Epoch 4/50
20/20 [==============================] - 3s 167ms/step - batch: 9.5000 - size: 5.9500 - loss: 1.9903 - accuracy: 0.2773 - val_loss: 2.0691 - val_accuracy: 0.3585
Epoch 5/50
20/20 [==============================] - 3s 167ms/step - batch: 9.5000 - size: 5.9500 - loss: 1.7656 - accuracy: 0.3613 - val_loss: 1.9672 - val_accuracy: 0.4340
Epoch 6/50
20/20 [==============================] - 3s 164ms/step - batch: 9.5000 - size: 5.9500 - loss: 1.4228 - accuracy: 0.5546 - val_loss: 2.5

In [16]:
classifier_json=classifier.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(classifier_json)
# serialize weights to HDF5
    classifier.save_weights("my_model_weights.h5")
    classifier.save("model.h5")
    print("Saved model to disk")

Saved model to disk



<br>
import cv2<br>
from matplotlib import pyplot as plt<br>
import os<br>
import numpy as np<br>
img = cv2.imread('C:/Users/Madhuri/AppData/Local/Programs/Python/Python38/Leaf_disease/data/d (7)_iaip.jpg')<br>
img_resize = cv2.resize(img, (128,128))<br>
CV2 reads an image in BGR format. We need to convert it to RGB<br>
b,g,r = cv2.split(img_resize)       # get b,g,r<br>
rgb_img = cv2.merge([r,g,b])     # switch it to rgb<br>
plt.imshow(rgb_img)<br>
label_map = (training_set.class_indices)<br>
print(label_map)<br>
img_rank4 = np.expand_dims(rgb_img/255, axis=0)<br>
classifier.predict(img_rank4)<br>
h = list(label_map.keys())[classifier.predict_classes(img_rank4)[0]]<br>
font = cv2.FONT_HERSHEY_DUPLEX<br>
cv2.putText(img, h, (10, 30), font, 1.0, (0, 0, 255), 1)<br>
cv2.imshow(h,img)<br>
print(h)<br>
